In [1]:
# reading ERA5 3-hourly files, running through a year,
#  and outputting a mean grid at its native resolution as downloaded
# 2/1/18 fixed bug in calculation of daytime-only zonal means
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap #, cm
import glob

In [2]:
ecdir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/'
yr = '15' # year 20x
#dn = 'night' # day/night flag, new 2/1/18 ---- GOT RID OF THIS AND SUPPPLANTED BY APPROACH BELOW
de = 3 # time delta, in hours, around midday/midnight. i.e. 6 means 6-18/18-6, 3 = 9-15/21-3
#ctr= 1.5 # local solar time to center on i.e. noon=12, 1:30pm=13.5
ctr= 13.5 # local solar time to center on i.e. noon=12, 1:30pm=13.5
ts = 3 # time step in hrs of data (cant be changed--downloaded this way)
mons = sorted(glob.glob(ecdir+yr+'*/'))
print(len(mons))
ordr = [11,0,1,2,3,4,5,6,7,8,9,10]
mons = [mons[i] for i in ordr]
#print(mons) # run in order DJF/MMA/JJA/SON/
hrs = str(ctr)+'_'+str(ctr-de)+'to'+str(ctr+de) #'0to24'
print('hrs: ',hrs)

12
hrs:  13.5_10.5to16.5


In [3]:
gsize = 0.5 # in degrees, normal grid (downloaded this way. if wanting to change output grid, do elsewhere)
latmax= 90  # NS latitude limits of grid (downloaded this way)
nx, ny = int(360.0/gsize), int(2.0*latmax/gsize)+1 
# EC has points AT lat/lon intersections, so EC data actually -180 to 179.5, -90 to 90
mean_grid_iwp = np.zeros([ny, nx])
mean_grid_swp = np.zeros([ny, nx])
zonal_mean_iwp = np.zeros(ny)
zonal_mean_swp = np.zeros(ny)
count = 0 # initialize
ndays = 0
#iwpall = []
print(int(nx/2),'nx: ',nx)

360 nx:  720


In [4]:
# read in landmask for era5:
#lsfile = ecdir+'land0.5deg.nc' # make sure size matches gsize above
#lf = Dataset(lsfile,'r')
#lmask = np.array(lf['lsm'])[0,:,:] # 0.0 for ocean, 1.0 for land

In [8]:
for m in mons:#[0:1]:
    mo = m[len(m)-3:len(m)-1]
    print(mo)
    iwpall = []
    ndays = 0
    if mo == '12' or mo == '03' or mo == '06' or mo == '09': #reset for each new season
        mean_grid_iwps = np.zeros([ny, nx])
        mean_grid_swps = np.zeros([ny, nx])
        counts = 0 # initialize
        ndayss = 0
        if mo=='12': seas = 'DJF'
        if mo=='03': seas = 'MAM'
        if mo=='06': seas = 'JJA'
        if mo=='09': seas = 'SON'
        
    days = sorted(os.listdir(m)) # all daily files in the month:
    print('starting mon: ',m)
    for d in days:#[0:1]:
        f = Dataset(m+d,'r')
        iwp, swp = np.array(f['tciw']), np.array(f['tcsw'])  #"total column ice/snow water" in kg/m^2
        #iwp[iwp > 0.2] = 0.2
        #swp[swp > 1.0] = 1.0 # A TEST FOR V2________ BE CAREFUL IF RUNNING LATER
        ndays += 1 # count days processed
        ndayss+= 1 # counts days processed
        for t in range(np.shape(iwp)[0]):
                
            subz = ((ctr+12)-t*ts)/24.0 * nx #)*((t*ts)/24.0) #+ nx 
            sub1, sub2 = int( subz - nx*de/24.0), int( subz + nx*de/24.0)
            
            if sub1 < 0 and sub2 < 0: sub1, sub2 = sub1+nx, sub2+nx
            #print(sub1,sub2)
            if sub1 < 0 or sub2 > nx:
                if sub1 < 0:  sub1 += nx
                if sub2 > nx: sub2 -= nx
                #print('oddz: ',sub1,sub2)
                iwp[t,:,sub1:sub2], swp[t,:,sub1:sub2] = np.nan, np.nan #for zonal means calc
                mean_grid_iwp[:,0:sub2] += iwp[t,:,0:sub2]
                mean_grid_swp[:,0:sub2] += swp[t,:,0:sub2]
                mean_grid_iwp[:,sub1:nx] += iwp[t,:,sub1:nx]
                mean_grid_swp[:,sub1:nx] += swp[t,:,sub1:nx]
                mean_grid_iwps[:,0:sub2] += iwp[t,:,0:sub2]
                mean_grid_swps[:,0:sub2] += swp[t,:,0:sub2]
                mean_grid_iwps[:,sub1:nx] += iwp[t,:,sub1:nx]
                mean_grid_swps[:,sub1:nx] += swp[t,:,sub1:nx]
                zonal_mean_iwp[:] += np.nanmean(iwp[t,:,:],axis=1) #dont count 
                zonal_mean_swp[:] += np.nanmean(swp[t,:,:],axis=1) #non-day pts
            else:
                mean_grid_iwp[:,sub1:sub2] += iwp[t,:,sub1:sub2]
                mean_grid_swp[:,sub1:sub2] += swp[t,:,sub1:sub2]
                mean_grid_iwps[:,sub1:sub2] += iwp[t,:,sub1:sub2]
                mean_grid_swps[:,sub1:sub2] += swp[t,:,sub1:sub2]
                zonal_mean_iwp[:] += iwp[t,:,sub1:sub2].mean(axis=1)
                zonal_mean_swp[:] += swp[t,:,sub1:sub2].mean(axis=1)
                
            count += 1
            counts += 1
            
            # new feb 2018, taking a-train timed slices for a histogram:
            #totice =   #### actually, just all time steps for now!
            #totice = np.reshape(iwp[t,:,:],[ny,nx])
            totice = np.reshape(iwp[t,:,:] + swp[t,:,:],[ny,nx])
            total_ice = np.reshape(totice[60:300,:],nx*240) # just 60NS into 1D 
            iwpall.extend(total_ice)
            ##iwpi.extend(np.reshape(iwp[t,:,:],[ny,nx]))#[lmask==0])
            ##iwps.extend(np.reshape(swp[t,:,:],[ny,nx]))#[lmask==0])
            
    
    nds = str(ndays)
    binz = [2**x for x in range(-14,6)]
    binzz = [-.001,2**(-14)]
    hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
    histaz, bin_edges = np.histogram(np.array(iwpall),bins=binzz)
    print('histaz',histaz,hista)
    fin = 'grids/era5.histo.all.v2.'+yr+mo+'.'+nds
    #fin = 'grids/era5.histo.all.ciwonly.v2.'+yr+mo+'.'+nds
    outz = np.append(histaz,hista)
    np.save(fin,outz) #hista)
            
    # if wanting seasonal total/count grids, uncomment save statements below
    if mo == '02' or mo == '05' or mo == '08' or mo == '11': #write out at end of each 3mo season
        mean_grid_iwps /= float(counts/(24.0/(de*2)))
        mean_grid_swps /= float(counts/(24.0/(de*2)))
        nds = str(ndayss)
        fin = 'grids/era5.'+seas+'.'+str(nx)+'x.'+str(ny)+'y.'+hrs+'_3hrly.iwp.swp.v1.'+yr+'.'+nds
        #fin = 'grids/era5.'+seas+'.'+str(nx)+'x.'+str(ny)+'y.'+hrs+'_3hrly.iwp.swp.v2.'+yr+'.'+nds
        #np.save(fin+'.grid',mean_grid_iwps)
        #np.save(fin+'.sgrid',mean_grid_swps)

mean_grid_iwp /= float(count/(24.0/(de*2)))
mean_grid_swp /= float(count/(24.0/(de*2)))
zonal_mean_iwp /= float(count)
zonal_mean_swp /= float(count)

12
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/1512/
histaz [14100680] [1135707 1161872 1296304 1505416 1720411 2145361 2649040 3193791 3371266
 3161913 2602569 1976652 1376137  853223  417756  140660   34105    9204
    2310]
01
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/1501/
histaz [15241661] [1010453 1116143 1250836 1446505 1660243 2061376 2541743 3029320 3158643
 2952499 2524163 1961266 1405041  872107  436704  144768   32763    7021
    1135]
02
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/1502/
histaz [13527787] [ 969305 1053391 1132028 1348008 1541598 1923818 2331980 2808530 2915479
 2677709 2254113 1735441 1222860  756590  364934  113977   23409    5081
    1156]
03
starting mon:  /home/dudavid/Dendrite/Dendrite/UserAreas/Dave/EC/1503/
histaz [14559691] [1017185 1111751 1324210 1471267 1727827 2146531 2621672 3166207 3349240
 3017870 2519928 1960528 1401895  872504  422956  128976   27196    5801
    1155]
04
start

In [6]:
# create histogram of IWP
#iwpa = np.array(iwpall)
binz = [2**x for x in range(-14,5)]

#print(iwpa.max(),iwpa.min())
#hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
#histi, bin_edges  = np.histogram(np.array(iwpi),bins=binz)
#hists, bin_edges  = np.histogram(np.array(iwps),bins=binz)

In [7]:
nd = str(ndays)
# dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
#if dn == 'night': hrs = '18to6'
#if dn == 'day': hrs = '6to18'
fin = 'grids/era5.'+str(nx)+'x.'+str(ny)+'y.'+hrs+'_3hrly.iwp.swp.v2.'+yr+'.'+nd
np.save(fin+'.grid',mean_grid_iwp)
np.save(fin+'.zonal',zonal_mean_iwp)
np.save(fin+'.sgrid',mean_grid_swp)
np.save(fin+'.szonal',zonal_mean_swp)

In [8]:
#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.plot(binz[1:],hista/sum(hista),'r-o',label='SWP+CIWP')
#plt.plot(binz[1:],histi/sum(histi),'b-o',label='Cloud ice')
#plt.plot(binz[1:],hists/sum(hists),'k-o',label='Snow')
#plt.xscale('log')
#plt.xlabel(r'$IWP [kg/m^2]$')
#plt.ylabel('Fraction of cloudy counts')
#plt.title('Oceanic IWP and SWP, '+mon+' ERA5')#dire)
#plt.ylim(.005,.16)
#plt.legend()
#plt.show()
#fig.savefig('img/era5.'+mon+'.histo.png')

In [9]:
# plot zonal means
figz = plt.figure(figsize=[7,7])
zax = np.linspace(-1*latmax,latmax,ny)
#plt.plot(1000*zonal_mean_iwp[::-1]+zonal_mean_swp[::-1],zax,'r-o',label='SWP+CIWP')
#plt.plot(1000*zonal_mean_iwp[::-1],zax,'b-o',label='CIWP')
#plt.plot(1000*zonal_mean_swp[::-1],zax,'g-o',label='SWP')
#plt.plot(1000*(zonal_mean_swp[::-1]+zonal_mean_iwp[::-1]),zax,'k-o',label='CIWP+SWP')
plt.plot(zonal_mean_iwp[::-1],zax,'b-o',label='CIWP')
plt.plot(zonal_mean_swp[::-1],zax,'g-o',label='SWP')
plt.plot((zonal_mean_swp[::-1]+zonal_mean_iwp[::-1]),zax,'k-o',label='CIWP+SWP')
plt.xlabel(r'$IWP [kg/m^2]$')
plt.ylabel(r'$Latitude [^oN]$')
plt.title('ERA5 '+dn+' 20'+yr+' zonal means')
plt.legend()
plt.show()
figz.savefig('img/era5.'+dn+'.20'+yr+'.day.zonal.jpg',dpi=400)

NameError: name 'dn' is not defined

In [ ]:
# plot global mean maps of CIWP/SWP
fig1 = plt.figure(figsize=[11,5])
print(info(mean_grid_iwp))
grdmap(mean_grid_iwp,-180,latmax,.001,0.2,1,'ERA5 '+dn+' 20'+yr+' CIWP mean')#+mon)
#grdmap(mean_grid_iwp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1,'ERA5 IWP')
fig1.savefig('img/era5.20'+yr+'.'+dn+'.ciwp_grid.jpg',dpi=500)
fig2 = plt.figure(figsize=[11,5])
print(info(mean_grid_swp))
grdmap(mean_grid_swp,-180,latmax,.001,0.2,1,'ERA5 '+dn+' 20'+yr+' SWP mean')#+mon)
#grdmap(mean_grid_swp,lomin=-180,lamin=70,vmin=.001,vmax=0.4,expflag=1)
fig2.savefig('img/era5.20'+yr+'.'+dn+'.swp_grid.jpg',dpi=500)